# Spotify Song Popularity Predictor and Genre Recommendation System

#### Go with THIS flow 

5. Recommendation system

Content-based filtering and/or Collaborative filtering 

Song recommender based on features of a song that is input by the user 

Return playlist of 10-20 similar song s

In [1]:
# !pip install spotipy

In [2]:
import pandas as pd
import numpy as np
import random
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

%matplotlib inline

## 5. Song Recommender Model 

In [3]:
df_recommender = pd.read_csv("../datasets/data_recommender.csv")
df_recommender.head()

,acousticness,artists,danceability,energy,explicit,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year,duration_mins
0,0.991000,['Mamie Smith'],0.598,0.224,0,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,0.12,0.0936,149.976,0.6340,1920,2.81
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,0.517,0,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,0.07,0.0534,86.889,0.9500,1920,2.50
2,0.993000,['Mamie Smith'],0.647,0.186,0,0.000018,0,0.5190,-12.098,1,Golfing Papa,0.04,0.1740,97.600,0.6890,1920,2.73
3,0.000173,['Oscar Velazquez'],0.730,0.798,0,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,0.17,0.0425,127.997,0.0422,1920,7.03
4,0.295000,['Mixe'],0.704,0.707,1,0.000246,10,0.4020,-6.036,0,Xuniverxe,0.02,0.0768,122.076,0.2990,1920,2.75


In [4]:
df_recommender.shape

(159477, 17)

### 5.1 Preprocessing

Since the dataframe still contains almost 160,000 rows, it's not feasible to create a recommender with all those songs. Instead, we'll be taking a subset of the dataframe with songs from 1990 onwards. In addition, all duplicated song versions from the SAME artist will be dropped, keeping only the version with the highest popularity. 

We will still keep the same song sung by different artist to ensure that songs with identical titles are not deleted. 

In [5]:
df_recommender = df_recommender.sort_values('popularity', ascending=False)
df_recommender.drop_duplicates(subset=['artists', 'name'], keep='first', inplace=True)

In [6]:
df_recommender_from_1990 = df_recommender[df_recommender.year >= 2000]
df_recommender_from_1990.describe()['year']

count    36524.000000
mean      2011.930265
std          6.119169
min       2000.000000
25%       2007.000000
50%       2013.000000
75%       2017.000000
max       2021.000000
Name: year, dtype: float64

In [7]:
df_recommender_from_1990.shape

(36524, 17)

In [8]:
df_recommender_from_1990[df_recommender_from_1990.name.duplicated()]['name'].value_counts().head(50)

2000 Years                                                     51
Happy New Year                                                 21
7 Years                                                        18
Stay                                                           16
New Year                                                       16
Year 2000                                                      15
Have Yourself a Merry Little Christmas                         15
2000 Light Years from Home                                     12
It's the Most Wonderful Time of the Year                       12
Home                                                           12
Neujahrsgruß / New Year's Address / Allocution du Nouvel An    12
Hold On                                                        11
2000 Years Ago                                                 10
Winter Wonderland                                              10
A Thousand Years                                               10
Intro     

In [9]:
drop_features = ['explicit', 'key', 'mode', 'year', 'duration_mins', 'artists']
scale_features = ['loudness', 'tempo']
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'speechiness', 'valence']

In [10]:
df_recommender_from_1990 = df_recommender_from_1990.drop(drop_features, axis=1)
df_recommender_from_1990

,acousticness,danceability,energy,instrumentalness,liveness,loudness,name,popularity,speechiness,tempo,valence
18818,0.721000,0.585,0.436,0.000013,0.1050,-8.761,drivers license,1.00,0.0601,143.874,0.1320
18639,0.221000,0.700,0.722,0.000000,0.2720,-3.558,Mood (feat. iann dior),0.96,0.0369,90.989,0.7560
18642,0.468000,0.737,0.802,0.000000,0.0931,-4.771,positions,0.96,0.0878,144.015,0.6820
18734,0.212000,0.863,0.666,0.000493,0.1030,-4.158,BICHOTA,0.95,0.1520,163.908,0.8380
18656,0.401000,0.731,0.573,0.000052,0.1130,-10.059,DÁKITI,0.95,0.0544,109.928,0.1450
...,...,...,...,...,...,...,...,...,...,...,...
113145,0.000951,0.772,0.546,0.924000,0.1020,-7.413,Force A,0.00,0.0543,124.000,0.0862
113158,0.007550,0.775,0.616,0.881000,0.1130,-8.415,Saff - Tex-Rec Remix,0.00,0.0579,124.989,0.0350
113160,0.076500,0.626,0.913,0.627000,0.0908,-9.882,Jump the Next,0.00,0.0370,95.995,0.4820
113154,0.001810,0.617,0.674,0.905000,0.1040,-9.088,Krim - Marco Asoleda vs Roman Kramer Remix,0.00,0.0541,124.996,0.1530


In [11]:
def min_max_scale(X, range=(0, 1)):
    mi, ma = range
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (ma - mi) + mi
    return X_scaled

In [12]:
for feature in scale_features: 
    df_recommender_from_1990[feature] = min_max_scale(df_recommender_from_1990[feature])
    print(df_recommender_from_1990[feature].describe())

count    36524.000000
mean         0.802669
std          0.078647
min          0.000000
25%          0.777480
50%          0.821710
75%          0.852051
max          1.000000
Name: loudness, dtype: float64
count    36524.000000
mean         0.553456
std          0.127978
min          0.000000
25%          0.451074
50%          0.563193
75%          0.627113
max          1.000000
Name: tempo, dtype: float64


In [13]:
df_recommender_from_1990 = df_recommender_from_1990.set_index('name', drop=True)
df_recommender_from_1990.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence
name,,,,,,,,,,
drivers license,0.721,0.585,0.436,0.000013,0.1050,0.789966,1.00,0.0601,0.648215,0.132
Mood (feat. iann dior),0.221,0.700,0.722,0.000000,0.2720,0.880072,0.96,0.0369,0.409945,0.756
positions,0.468,0.737,0.802,0.000000,0.0931,0.859065,0.96,0.0878,0.648851,0.682
BICHOTA,0.212,0.863,0.666,0.000493,0.1030,0.869681,0.95,0.1520,0.738477,0.838
DÁKITI,0.401,0.731,0.573,0.000052,0.1130,0.767487,0.95,0.0544,0.495274,0.145


##### Genres 

In [14]:
df_genres = pd.read_csv("../datasets/data_by_genres_cleaned.csv", index_col=['genres'])
df_genres.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,duration_mins
genres,,,,,,,,,,,,,
21st century classical,0.754600,0.284100,0.159580,0.484374,0.168580,-22.153400,0.062060,91.351000,0.143380,0.066000,4,1,5.88
432hz,0.485515,0.312000,0.391678,0.477250,0.265940,-18.131267,0.071717,118.900933,0.236483,0.412000,11,1,17.46
8-bit,0.028900,0.673000,0.950000,0.630000,0.069000,-7.899000,0.292000,192.816000,0.997000,0.000000,5,1,2.22
[],0.535793,0.546937,0.485430,0.278442,0.220970,-11.624754,0.101511,116.068980,0.486361,0.123508,7,1,4.16
a cappella,0.694276,0.516172,0.330533,0.036080,0.222983,-12.656547,0.083627,105.506031,0.454077,0.390862,7,1,3.36


In [15]:
df_genres.drop(['key', 'mode', 'duration_mins'], axis=1, inplace=True)

In [16]:
df_genres.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
genres,,,,,,,,,,
21st century classical,0.754600,0.284100,0.159580,0.484374,0.168580,-22.153400,0.062060,91.351000,0.143380,0.066000
432hz,0.485515,0.312000,0.391678,0.477250,0.265940,-18.131267,0.071717,118.900933,0.236483,0.412000
8-bit,0.028900,0.673000,0.950000,0.630000,0.069000,-7.899000,0.292000,192.816000,0.997000,0.000000
[],0.535793,0.546937,0.485430,0.278442,0.220970,-11.624754,0.101511,116.068980,0.486361,0.123508
a cappella,0.694276,0.516172,0.330533,0.036080,0.222983,-12.656547,0.083627,105.506031,0.454077,0.390862


Re-organising columns

In [17]:
column_order = list(df_recommender_from_1990.columns)
df_genres = df_genres[column_order]
df_genres.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence
genres,,,,,,,,,,
21st century classical,0.754600,0.284100,0.159580,0.484374,0.168580,-22.153400,0.066000,0.062060,91.351000,0.143380
432hz,0.485515,0.312000,0.391678,0.477250,0.265940,-18.131267,0.412000,0.071717,118.900933,0.236483
8-bit,0.028900,0.673000,0.950000,0.630000,0.069000,-7.899000,0.000000,0.292000,192.816000,0.997000
[],0.535793,0.546937,0.485430,0.278442,0.220970,-11.624754,0.123508,0.101511,116.068980,0.486361
a cappella,0.694276,0.516172,0.330533,0.036080,0.222983,-12.656547,0.390862,0.083627,105.506031,0.454077


In [18]:
df_genres.sort_values('popularity', ascending=False).head(50)

,acousticness,danceability,energy,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence
genres,,,,,,,,,,
chinese electropop,0.002570,0.660000,0.787000,0.000000e+00,0.323000,-4.592000,0.790000,0.032000,142.018000,0.199000
korean mask singer,0.238000,0.658000,0.749000,2.030000e-06,0.272000,-2.967000,0.780000,0.063400,128.909000,0.442000
yaoi,0.006550,0.603000,0.964000,3.040000e-06,0.143000,-2.886000,0.770000,0.048700,135.028000,0.796000
dutch rap pop,0.216000,0.767000,0.643000,1.040000e-04,0.080600,-5.225000,0.770000,0.101000,169.990000,0.536000
rochester mn indie,0.034700,0.655000,0.487000,8.970000e-01,0.271000,-7.988000,0.760000,0.033000,139.914000,0.045400
dong-yo,0.259000,0.829000,0.886000,0.000000e+00,0.055900,-1.746000,0.760000,0.112000,115.056000,0.777000
afroswing,0.171333,0.758333,0.512000,3.273333e-05,0.101033,-7.562667,0.753333,0.211000,102.061000,0.480333
j-rap,0.096100,0.648000,0.904000,2.070000e-06,0.062900,-4.565000,0.750000,0.078600,121.994000,0.728000
estonian pop,0.285000,0.458000,0.696000,0.000000e+00,0.123000,-4.742000,0.750000,0.040200,174.141000,0.418000


### 5.2 Cosine Similarity 

In [19]:
sim_matrix = cosine_similarity(df_recommender_from_1990)
songs_sim = pd.DataFrame(sim_matrix, columns=df_recommender_from_1990.index, index=df_recommender_from_1990.index)
songs_sim

name,drivers license,Mood (feat. iann dior),positions,BICHOTA,DÁKITI,WITHOUT YOU,Whoopty,Therefore I Am,34+35,LA NOCHE DE ANOCHE,...,Bionic Flame - Dark Room Alliance Obscure Rework,Step Asidex,Carretta - David Carretta Remix,Coupe Drug,Tokio Beatz,Force A,Saff - Tex-Rec Remix,Jump the Next,Krim - Marco Asoleda vs Roman Kramer Remix,London Groove
name,,,,,,,,,,,,,,,,,,,,,
drivers license,1.000000,0.878134,0.931726,0.884042,0.972977,0.927943,0.892633,0.878563,0.922586,0.873750,...,0.578682,0.713137,0.567673,0.608904,0.614280,0.587354,0.596418,0.619886,0.584717,0.557085
Mood (feat. iann dior),0.878134,1.000000,0.979286,0.978888,0.932400,0.978757,0.984882,0.963889,0.983186,0.956811,...,0.608401,0.759808,0.729584,0.723735,0.706965,0.630964,0.634664,0.767626,0.648837,0.622433
positions,0.931726,0.979286,1.000000,0.983522,0.958440,0.970867,0.983919,0.953432,0.978052,0.944472,...,0.632146,0.797853,0.734481,0.748842,0.707305,0.649252,0.656206,0.775245,0.667680,0.634459
BICHOTA,0.884042,0.978888,0.983522,1.000000,0.929166,0.972536,0.997247,0.974418,0.978838,0.956691,...,0.641070,0.793654,0.755734,0.766269,0.711927,0.662556,0.664010,0.774183,0.673132,0.618459
DÁKITI,0.972977,0.932400,0.958440,0.929166,1.000000,0.967560,0.935282,0.924490,0.972974,0.951624,...,0.637899,0.755515,0.639207,0.666303,0.696033,0.651916,0.664149,0.697963,0.648148,0.611313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Force A,0.587354,0.630964,0.649252,0.662556,0.651916,0.630802,0.643617,0.659899,0.668728,0.670819,...,0.998053,0.943165,0.930378,0.929951,0.967260,1.000000,0.998198,0.921331,0.991652,0.950804
Saff - Tex-Rec Remix,0.596418,0.634664,0.656206,0.664010,0.664149,0.632017,0.645269,0.653403,0.674663,0.678096,...,0.997342,0.944936,0.925464,0.931244,0.974596,0.998198,1.000000,0.925977,0.992119,0.952149
Jump the Next,0.619886,0.767626,0.775245,0.774183,0.697963,0.705198,0.755401,0.728794,0.749580,0.741885,...,0.912724,0.960216,0.980555,0.968289,0.953899,0.921331,0.925977,1.000000,0.949535,0.933344


In [20]:
songs_sim['drivers license'].sort_values(ascending=False).head(20)

name
drivers license                  1.000000
you broke me first               0.995865
The Scientist                    0.994393
Hope                             0.993651
Photograph                       0.993509
Give Me Love                     0.992375
If I Ain't Got You               0.992303
Nikes                            0.991786
If You Want Love                 0.991230
ocean eyes                       0.991230
Lose You To Love Me              0.991089
Moral of the Story               0.990850
Runaway                          0.990314
Control                          0.990103
I Won't Give Up                  0.989875
Another Love                     0.989712
Corduroy Dreams                  0.989556
Hold On                          0.988616
Get You The Moon (feat. Snøw)    0.988511
Follow The Sun                   0.988474
Name: drivers license, dtype: float64

#### Genre(s) To Recommend

In [21]:
genres_to_recommend = ['swedish tropical house', 'indie rockism' , 'lo-fi chill', 'rochester mn indie']

In [22]:
genre_choice = 'rochester mn indie'

Let's recommend songs with similar features to swedish tropical house

In [23]:
df_genre_recc = pd.concat([pd.DataFrame(df_genres.loc[genre_choice]).T, df_recommender_from_1990], axis=0)
df_genre_recc.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence
rochester mn indie,0.0347,0.655,0.487,0.897000,0.2710,-7.988000,0.76,0.0330,139.914000,0.0454
drivers license,0.7210,0.585,0.436,0.000013,0.1050,0.789966,1.00,0.0601,0.648215,0.1320
Mood (feat. iann dior),0.2210,0.700,0.722,0.000000,0.2720,0.880072,0.96,0.0369,0.409945,0.7560
positions,0.4680,0.737,0.802,0.000000,0.0931,0.859065,0.96,0.0878,0.648851,0.6820
BICHOTA,0.2120,0.863,0.666,0.000493,0.1030,0.869681,0.95,0.1520,0.738477,0.8380


In [24]:
start_time = datetime.now()
print(f"Recommending {genre_choice} music...")
sim_matrix_genre = cosine_similarity(df_genre_recc)
songs_sim_genre = pd.DataFrame(sim_matrix_genre, columns=df_genre_recc.index, index=df_genre_recc.index)
songs_sim_genre

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Recommending rochester mn indie music...
Duration: 0:06:11.720545


In [25]:
df_songs_sim_genre = pd.DataFrame(sim_matrix_genre, columns=df_genre_recc.index, index=df_genre_recc.index)
df_songs_sim_genre

,rochester mn indie,drivers license,Mood (feat. iann dior),positions,BICHOTA,DÁKITI,WITHOUT YOU,Whoopty,Therefore I Am,34+35,...,Bionic Flame - Dark Room Alliance Obscure Rework,Step Asidex,Carretta - David Carretta Remix,Coupe Drug,Tokio Beatz,Force A,Saff - Tex-Rec Remix,Jump the Next,Krim - Marco Asoleda vs Roman Kramer Remix,London Groove
rochester mn indie,1.000000,0.346128,0.196277,0.306634,0.342519,0.276024,0.236306,0.321230,0.217008,0.256247,...,0.342489,0.334431,0.296641,0.458771,0.273827,0.315849,0.320175,0.244820,0.326477,0.219887
drivers license,0.346128,1.000000,0.878134,0.931726,0.884042,0.972977,0.927943,0.892633,0.878563,0.922586,...,0.578682,0.713137,0.567673,0.608904,0.614280,0.587354,0.596418,0.619886,0.584717,0.557085
Mood (feat. iann dior),0.196277,0.878134,1.000000,0.979286,0.978888,0.932400,0.978757,0.984882,0.963889,0.983186,...,0.608401,0.759808,0.729584,0.723735,0.706965,0.630964,0.634664,0.767626,0.648837,0.622433
positions,0.306634,0.931726,0.979286,1.000000,0.983522,0.958440,0.970867,0.983919,0.953432,0.978052,...,0.632146,0.797853,0.734481,0.748842,0.707305,0.649252,0.656206,0.775245,0.667680,0.634459
BICHOTA,0.342519,0.884042,0.978888,0.983522,1.000000,0.929166,0.972536,0.997247,0.974418,0.978838,...,0.641070,0.793654,0.755734,0.766269,0.711927,0.662556,0.664010,0.774183,0.673132,0.618459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Force A,0.315849,0.587354,0.630964,0.649252,0.662556,0.651916,0.630802,0.643617,0.659899,0.668728,...,0.998053,0.943165,0.930378,0.929951,0.967260,1.000000,0.998198,0.921331,0.991652,0.950804
Saff - Tex-Rec Remix,0.320175,0.596418,0.634664,0.656206,0.664010,0.664149,0.632017,0.645269,0.653403,0.674663,...,0.997342,0.944936,0.925464,0.931244,0.974596,0.998198,1.000000,0.925977,0.992119,0.952149
Jump the Next,0.244820,0.619886,0.767626,0.775245,0.774183,0.697963,0.705198,0.755401,0.728794,0.749580,...,0.912724,0.960216,0.980555,0.968289,0.953899,0.921331,0.925977,1.000000,0.949535,0.933344
Krim - Marco Asoleda vs Roman Kramer Remix,0.326477,0.584717,0.648837,0.667680,0.673132,0.648148,0.629712,0.656480,0.650489,0.669049,...,0.992481,0.944762,0.953340,0.957069,0.972954,0.991652,0.992119,0.949535,1.000000,0.975088


Recommend Swedish Tropical House

In [26]:
df_songs_sim_genre[genre_choice].sort_values(ascending=False)[1:31]

The Arrival - 2015 Remaster                                            0.671297
Year 2019                                                              0.659593
Nucleus                                                                0.651140
Glimpse of a Dream (2019)                                              0.646052
Frederick's Bass Tester #16                                            0.645305
Kitty - 2013 Mix                                                       0.644866
Return 2 Dust                                                          0.634321
Streetlights                                                           0.632911
Since 2016 Didn't Kill Me (Next Year Might...)                         0.619598
Weißes Rauschen: Baby Schlafhilfe                                      0.616521
A State Of Trance (ASOT 997) - ASOT Year Mix 2020 Outro                0.613270
Faithful Servant                                                       0.609140
Drive-In Saturday - German Single Edit; 

In [27]:
song_list = list(df_songs_sim_genre[genre_choice].sort_values(ascending=False)[1:31].index)
song_list

['The Arrival - 2015 Remaster',
 'Year 2019',
 'Nucleus',
 'Glimpse of a Dream (2019)',
 "Frederick's Bass Tester #16",
 'Kitty - 2013 Mix',
 'Return 2 Dust',
 'Streetlights',
 "Since 2016 Didn't Kill Me (Next Year Might...)",
 'Weißes Rauschen: Baby Schlafhilfe',
 'A State Of Trance (ASOT 997) - ASOT Year Mix 2020 Outro',
 'Faithful Servant',
 'Drive-In Saturday - German Single Edit; 2015 Remaster',
 'Into the Flame - 2012 Remaster',
 "It's the Most Wonderful Time of the Year",
 'Tibbie Dunbar (mog) - 2011 Remaster',
 'Year of the Dog',
 'Evermore',
 "Entr'acte-Valse",
 "Frederick's Bass Tester #19",
 'The Many Things That Life Could Bring - Live 2012',
 'The Years All Pass Me By',
 'The Milagro Beanfield War: Theme - From "The Milagro Beanfield War"',
 'The Moth (Medley) - 2004 Remaster',
 'The Winter Ends - Demo January 2013',
 'Stop and Stare',
 'Misery, Shining Like a Bird (Demo 2010)',
 'When a Man Loves a Woman',
 'Everything',
 'Down the Garden Path']

### 5.3 Nearest Neighbors

In [28]:
# from sklearn.neighbors import NearestNeighbors

In [29]:
# nn = NearestNeighbors(n_neighbors=10)
# nn.fit(df_recommender2)
# distances, indices = nn.kneighbors(df_recommender2)

### 5.4 Spotify API 

In [30]:
def get_song():
    song_chosen = False

    while not song_chosen: 
        song_choice = input("What song sparks joy in your life?: ")
        try: 
            song_query = df_recommender2.loc[song_choice]
        except KeyError: 
            song_choice = ""
            print("Sorry! That song is not in our limited dataset. Please choose another song")
            song_chosen = False
        if song_choice != "":
            song_chosen = True
    return song_choice
    

In [31]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [32]:
CLIENT_ID = "da356cb5fb1c4b3b9546ada6dd4dbd12"
CLIENT_SECRET = "e50029f63a2e4cf589f0f5ec903af115"

In [33]:
song_query = "champagne problems"

In [34]:
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                              client_secret=CLIENT_SECRET,
                              redirect_uri="http://example.com",
                              scope="playlist-modify-public",
                              show_dialog=True,
                             cache_path="token.txt"))
user_id = sp.current_user()["id"]

In [35]:
# top_20_songs = list(songs_sim[song_query].sort_values(ascending=False).head(20).index)
# top_20_songs

In [36]:
def find_song(top_20_songs):
    song_uris = []
    for song in top_20_songs: 
        result = sp.search(q=f"track:{song}", type="track")
        try: 
            uri = result['tracks']['items'][0]['uri']
            song_uris.append(uri)
        except IndexError:
            print(f"{song} doesn't exist in Spotify. Skipped.")
    return song_uris

In [37]:
find_song(genre_choice)

['spotify:track:2VJOpzv5sBpstCX9venJr5',
 'spotify:track:7MAibcTli4IisCtbHKrGMh',
 'spotify:track:4iJyoBOLtHqaGxP12qzhQI',
 'spotify:track:3FAJ6O0NOHQV8Mc5Ri6ENp',
 'spotify:track:4iJyoBOLtHqaGxP12qzhQI',
 'spotify:track:5QO79kh1waicV47BqGRL3g',
 'spotify:track:7MAibcTli4IisCtbHKrGMh',
 'spotify:track:4iJyoBOLtHqaGxP12qzhQI',
 'spotify:track:2VJOpzv5sBpstCX9venJr5',
 'spotify:track:6Pf1Y9DjaDLp53XyLXpaSg',
 'spotify:track:6fRxMU4LWwyaSSowV441IU',
 'spotify:track:67BtfxlNbhBmCDR2L2l8qd',
 'spotify:track:6Pf1Y9DjaDLp53XyLXpaSg',
 'spotify:track:3Ofmpyhv5UAQ70mENzB277',
 'spotify:track:67BtfxlNbhBmCDR2L2l8qd',
 'spotify:track:7MAibcTli4IisCtbHKrGMh',
 'spotify:track:3Ofmpyhv5UAQ70mENzB277',
 'spotify:track:4iJyoBOLtHqaGxP12qzhQI']

In [38]:
playlist_name = f"Your {genre_choice.title()} Playlist"
playlist = sp.user_playlist_create(user='1184735198', name=playlist_name)
sp.playlist_add_items(playlist_id=playlist['id'], items=find_song(genre_choice))

{'snapshot_id': 'Miw3MjcxYzg4MjZmMmE2MDk0MzdjYjBhOWFkMGJiNjM2ODI4NTBiMTRi'}